In [1]:
import pandas as pd
import numpy as np

import string
from re import sub

import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.preprocessing import LabelEncoder
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
from keras.layers import Input

In [3]:
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix


In [4]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))

In [5]:
y_train = y_train[slice(0, len(y_train), 3)]
x_train = x_train[slice(0, len(x_train), 3)]

In [6]:
y_test_np = []

for y in y_test:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_test_np.append(l)
    
y_test_np = np.asarray(y_test_np)
y_test_np.shape

(70000, 14)

In [7]:
y_train_np = []

for y in y_train:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_np.append(l)
    
y_train_np = np.asarray(y_train_np)
y_train_np.shape

(186667, 14)

In [8]:
del y_train

In [9]:
import tensorflow as tf

from time import time

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1050 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 6.1



In [10]:
y_train_np.shape

(186667, 14)

In [11]:
x_train.shape

(186667, 1000)

In [12]:
# del x_train

In [13]:
callback = EarlyStopping(monitor='val_loss',patience=2, min_delta=0.0001)

checkpoint = ModelCheckpoint("best_model", monitor='loss', verbose=1, save_best_only=True, mode='min')

reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

In [14]:
model_l1 = Sequential()
# model_l1.add(Embedding(56000, 1000))
model_l1.add(Input(shape=(1000,), name='input'))
# model_l1.add(Conv1D(128, 5, activation='relu'))
# model_l1.add(GlobalMaxPooling1D()) # added
model_l1.add(Dense(128, activation='relu'))
model_l1.add(Dropout(0.1))
model_l1.add(Dense(32, activation='relu'))
model_l1.add(Dense(14, activation='softmax')) 
model_l1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1.summary()

history_l1 = model_l1.fit(x_train, y_train_np, epochs=10, verbose=1, validation_split=0.2, batch_size=32
                   ,callbacks=[reduce,callback,checkpoint])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               128128    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 14)                462       
                                                                 
Total params: 132,718
Trainable params: 132,718
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
4665/4667 [============================>.] - ETA: 0s - loss: 0.1385 - accuracy: 0.9612
Epoch 1: loss improved from inf to 0.13848, saving model to best_model
INFO:tensorfl

In [17]:
import sys
import gc
gc.collect()

18152

In [18]:
score_l1 = model_l1.evaluate(x_test, y_test_np, verbose=1)
print('Test score:', score_l1[0])
print('Test accuracy:', score_l1[1])

2188/2188 [==============================] - 32s 15ms/step - loss: 2.8494 - accuracy: 0.8354
Test score: 2.849395275115967
Test accuracy: 0.8354285955429077


In [20]:
y_pred = model_l1.predict(x_test)

2188/2188 [==============================] - 17s 8ms/step


In [22]:
confusion_matrix(y_test_np, y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [23]:
classification_report(y_test_np, y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [24]:
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test_np, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4650   55   98    8   18   90   58    4    2    5    4    8    0    0]
 [  40 4915    4    1    6    2   27    2    1    0    2    0    0    0]
 [  27   10 4855    9   34    1    7    0    1    1    3   52    0    0]
 [   7    3   27 4939   11    5    0    2    0    6    0    0    0    0]
 [   7   16  136   16 4801   13    7    2    0    1    0    1    0    0]
 [  31    0    2    2    0 4957    5    2    0    0    1    0    0    0]
 [  56   56   14    1    7   20 4802   34    9    1    0    0    0    0]
 [   3    1    4    0    1    1   22 4944   21    2    1    0    0    0]
 [   1    5    3    0    1    0   11   17 4961    0    0    1    0    0]
 [   2    0    1    1    0    0    0    4    0 4970   22    0    0    0]
 [  13    1    2    0    0    1    1    2    0   69 4911    0    0    0]
 [  26    0  194    1    0    3    1    0    0    0    0 4775    0    0]
 [1203   48 3177   13   10   97   36   16    3    8    4  385    0    0]
 [1321   66 3192   12   26   41   33    1    2    4